In [1]:
 
import json,csv

dir="./Pulled_Data/"
  
# Opening JSON file
f = open(dir+'country_codes.jsonl')

with open(dir+'country_codes.jsonl', 'r') as json_file:
    json_list = list(json_file)

output=[]
mapping_dict=dict()
for json_str in json_list:
    result = json.loads(json_str)
    if result['name']=="korea-republic-of":
        result['name']="south-korea"
    mapping_dict[result['name']]=result['iso_3']
    output.append([result['iso_3'],result['name']])

with open(dir+'country_codes.csv', 'w', encoding='UTF-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['iso_3','name'])
    writer.writerows(output)
    # print(f"result: {result}")
    # print(isinstance(result, dict))


In [2]:
import rltk
import csv
import datetime
import pandas as pd

# You can use this tokenizer in case you need to manipulate some data
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()
tokenizer

In [3]:

class country_codes(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ""
    @rltk.cached_property
    def id(self):
        return self.raw_object["name"]

    # @rltk.cached_property
    # def country_name(self):
    #     return self.raw_object["name"]
        


class FTA_inforce(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ""

    @rltk.cached_property
    def id(self):
        return self.raw_object["country"]

    # @rltk.cached_property
    # def country_name(self):
    #     return self.raw_object["country"]

class gdp_population(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ""

    @rltk.cached_property
    def id(self):
        return self.raw_object["country"]


    # @rltk.cached_property
    # def country_name(self):
    #     return self.raw_object["country"]


class yearly_gdp_world(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ""

    @rltk.cached_property
    def id(self):
        return self.raw_object["country"]

    # @rltk.cached_property
    # def country_name(self):
    #     return self.raw_object["country"]



In [4]:
dir_ = "./Pulled_Data/"
country_codes_ = dir_ + 'country_codes.csv'
fta_inforce_ = dir_ + 'fta_inforce.csv' 
gdp_population_= dir_ + 'gdp_population.csv' 
yearly_gdp_world_= dir_ + 'yearly_gdp_world.csv' 


country_codes_ds = rltk.Dataset(
    rltk.CSVReader(open(country_codes_, encoding="utf-8",newline='')), record_class=country_codes
)
fta_inforce_ds = rltk.Dataset(
    rltk.CSVReader(open(fta_inforce_, encoding="utf-8")), record_class=FTA_inforce
)
gdp_population_ds = rltk.Dataset(
    rltk.CSVReader(open(gdp_population_, encoding="utf-8")), record_class=gdp_population
)

yearly_gdp_world_ds = rltk.Dataset(
    rltk.CSVReader(open(yearly_gdp_world_, encoding="utf-8")), record_class=yearly_gdp_world
)

In [5]:
print(country_codes_ds.generate_dataframe().head(5))
print(fta_inforce_ds.generate_dataframe().head(5))
print(gdp_population_ds.generate_dataframe().head(5))
print(yearly_gdp_world_ds.generate_dataframe().head(5))
# print(country_codes_ds.get_record('AFG'))

               id
0     afghanistan
1   aland-islands
2         albania
3         algeria
4  american-samoa
            id
0       Panama
1     Colombia
2  South Korea
3         Peru
4   Costa Rica
              id
0  United States
1          China
2          Japan
3        Germany
4          India
                            id
0                        Aruba
1  Africa Eastern and Southern
2                  Afghanistan
3   Africa Western and Central
4                       Angola


In [6]:
bg2 = rltk.HashBlockGenerator()
block_fta_inforce = bg2.generate(

    bg2.block(country_codes_ds, function_=lambda r: (r.id[:1].lower())),
    bg2.block(fta_inforce_ds, function_=lambda r: (r.id[:1].lower())),
)

pairs_fta_inforce = rltk.get_record_pairs(country_codes_ds, fta_inforce_ds)#, block=block_fta_inforce)

count_pairs = 0
for r1, r2 in pairs_fta_inforce:
    count_pairs += 1

print(count_pairs)
print(
    "Total comparisons:", len(country_codes_ds.generate_dataframe()) * len(fta_inforce_ds.generate_dataframe())
)
print("Length of pairs", count_pairs)
print(
    "reduction ratio",
    count_pairs / (len(country_codes_ds.generate_dataframe()) * len(fta_inforce_ds.generate_dataframe())),
)

4980
Total comparisons: 4980
Length of pairs 4980
reduction ratio 1.0


In [7]:
bg2 = rltk.HashBlockGenerator()
block_gdp_population = bg2.generate(

    bg2.block(country_codes_ds, function_=lambda r: (r.id[:1].lower())),
    bg2.block(gdp_population_ds, function_=lambda r: (r.id[:1].lower())),
)

pairs_gdp_population = rltk.get_record_pairs(country_codes_ds, gdp_population_ds, block=block_gdp_population)

count_pairs = 0
for r1, r2 in pairs_gdp_population:
    count_pairs += 1

print(count_pairs)
print(
    "Total comparisons:", len(country_codes_ds.generate_dataframe()) * len(gdp_population_ds.generate_dataframe())
)
print("Length of pairs", count_pairs)
print(
    "reduction ratio",
    count_pairs / (len(country_codes_ds.generate_dataframe()) * len(gdp_population_ds.generate_dataframe())),
)

3154
Total comparisons: 47310
Length of pairs 3154
reduction ratio 0.06666666666666667


In [8]:
bg2 = rltk.HashBlockGenerator()
block_yearly_gdp_world = bg2.generate(

    bg2.block(country_codes_ds, function_=lambda r: (r.id[:1].lower())),
    bg2.block(yearly_gdp_world_ds, function_=lambda r: (r.id[:1].lower())),
)

pairs_yearly_gdp_world = rltk.get_record_pairs(country_codes_ds, yearly_gdp_world_ds, block=block_yearly_gdp_world)

count_pairs = 0
for r1, r2 in pairs_yearly_gdp_world:
    count_pairs += 1

print(count_pairs)
print(
    "Total comparisons:", len(country_codes_ds.generate_dataframe()) * len(yearly_gdp_world_ds.generate_dataframe())
)
print("Length of pairs", count_pairs)
print(
    "reduction ratio",
    count_pairs / (len(country_codes_ds.generate_dataframe()) * len(yearly_gdp_world_ds.generate_dataframe())),
)

4260
Total comparisons: 66234
Length of pairs 4260
reduction ratio 0.0643174200561645


In [9]:
def country_similarity_1(r1, r2):
    ''' Example dummy similiary function '''
    s1 = r1.id.lower()
    s2 = r2.id.lower()
    
    return rltk.jaro_winkler_similarity(s1, s2)
    # return rltk.needleman_wunsch_similarity(s1, s2)
    # return rltk.levenshtein_similarity(s1,s2)
    # return rltk.string_cosine_similarity(s1,s2)
    # return rltk.optimal_string_alignment_similarity(s1,s2)

In [10]:
# threshold value to determine if we are confident the record match
MY_TRESH = 0.8 # this number is just an example, you need to change it

# entity linkage scoring function
def rule_based_method(r1, r2):
    score_1 = country_similarity_1(r1, r2)
    # return two values: boolean if they match or not, float to determine confidence
    return score_1 > MY_TRESH, score_1

In [11]:
pairs_fta_inforce = rltk.get_record_pairs(country_codes_ds, fta_inforce_ds)#, block=block_gdp_population)
all_predictions=dict()
all_predictions_key=dict()
predictions = dict()
score=dict()
for id1, id2 in pairs_fta_inforce:
    result, confidence = rule_based_method(id1, id2)
    if id2.id not in predictions:
        predictions[id2.id]=""
        score[id2.id]=0.0
    if result:
        if confidence>score[id2.id]:
            predictions[id2.id]=id1.id
            score[id2.id]=confidence

count_nulls=0
not_found=[]
for k,v in predictions.items():
    if v=="":
        count_nulls+=1
        not_found.append(k)
    if k not in all_predictions.keys():
        all_predictions[k]=v
        if v in mapping_dict.keys():
            all_predictions_key[k]=mapping_dict[v]
        else:
            all_predictions_key[k]=""

len(predictions),len(fta_inforce_ds.generate_dataframe()),count_nulls,not_found,predictions

(20,
 20,
 0,
 [],
 {'Panama': 'panama',
  'Colombia': 'colombia',
  'South Korea': 'south-korea',
  'Peru': 'peru',
  'Costa Rica': 'costa-rica',
  'Oman': 'oman',
  'Dominican Republic': 'dominican-republic',
  'Bahrain': 'bahrain',
  'Guatemala': 'guatemala',
  'Nicaragua': 'nicaragua',
  'Honduras': 'honduras',
  'El Salvador': 'el-salvador',
  'Morocco': 'morocco',
  'Australia': 'australia',
  'Chile': 'chile',
  'Singapore': 'singapore',
  'Jordan': 'jordan',
  'Mexico': 'mexico',
  'Canada': 'canada',
  'Israel': 'israel'})

In [12]:
pairs_gdp_population = rltk.get_record_pairs(country_codes_ds, gdp_population_ds)#, block=block_gdp_population)
predictions = dict()
score=dict()
for id1, id2 in pairs_gdp_population:
    result, confidence = rule_based_method(id1, id2)
    if id2.id not in predictions:
        predictions[id2.id]=""
        score[id2.id]=0.0
    if result:
        if confidence>score[id2.id]:
            predictions[id2.id]=id1.id
            score[id2.id]=confidence

count_nulls=0
not_found=[]
for k,v in predictions.items():
    if v=="":
        count_nulls+=1
        not_found.append(k)
    if k not in all_predictions.keys():
        all_predictions[k]=v
        if v in mapping_dict.keys():
            all_predictions_key[k]=mapping_dict[v]
        else:
            all_predictions_key[k]=""

len(predictions),len(gdp_population_ds.generate_dataframe()),count_nulls,not_found,predictions


(190,
 190,
 6,
 ['DR Congo',
  'Laos',
  'State of Palestine',
  'North Macedonia',
  'Cabo Verde',
  'St. Vincent & Grenadines'],
 {'United States': 'united-states',
  'China': 'china',
  'Japan': 'japan',
  'Germany': 'germany',
  'India': 'india',
  'United Kingdom': 'united-kingdom',
  'France': 'france',
  'Brazil': 'brazil',
  'Italy': 'italy',
  'Canada': 'canada',
  'Russia': 'russian-federation',
  'South Korea': 'south-korea',
  'Australia': 'australia',
  'Spain': 'spain',
  'Mexico': 'mexico',
  'Indonesia': 'indonesia',
  'Turkey': 'turkey',
  'Netherlands': 'netherlands',
  'Saudi Arabia': 'saudi-arabia',
  'Switzerland': 'switzerland',
  'Argentina': 'argentina',
  'Sweden': 'sweden',
  'Poland': 'poland',
  'Belgium': 'belgium',
  'Thailand': 'thailand',
  'Iran': 'ireland',
  'Austria': 'austria',
  'Norway': 'norway',
  'United Arab Emirates': 'united-arab-emirates',
  'Nigeria': 'nigeria',
  'Israel': 'israel',
  'South Africa': 'south-africa',
  'Hong Kong': 'hong-

In [13]:
pairs_yearly_gdp_world = rltk.get_record_pairs(country_codes_ds, yearly_gdp_world_ds)#, block=block_gdp_population)
predictions = dict()
score=dict()
for id1, id2 in pairs_yearly_gdp_world:
    result, confidence = rule_based_method(id1, id2)
    if id2.id not in predictions:
        predictions[id2.id]=""
        score[id2.id]=0.0
    if result:
        if confidence>score[id2.id]:
            predictions[id2.id]=id1.id
            score[id2.id]=confidence

count_nulls=0
not_found=[]
for k,v in predictions.items():
    if v=="":
        count_nulls+=1
        not_found.append([k])
    if k not in all_predictions.keys():
        all_predictions[k]=v
        if v in mapping_dict.keys():
            all_predictions_key[k]=mapping_dict[v]
        else:
            all_predictions_key[k]=""

len(predictions),len(yearly_gdp_world_ds.generate_dataframe()),count_nulls,not_found,predictions



(266,
 266,
 59,
 [['Africa Eastern and Southern'],
  ['Africa Western and Central'],
  ['Arab World'],
  ['Channel Islands'],
  ['Cabo Verde'],
  ['Caribbean small states'],
  ['Czechia'],
  ['East Asia & Pacific (excluding high income)'],
  ['Early-demographic dividend'],
  ['East Asia & Pacific'],
  ['Europe & Central Asia (excluding high income)'],
  ['Europe & Central Asia'],
  ['Euro area'],
  ['European Union'],
  ['Fragile and conflict affected situations'],
  ['High income'],
  ['Heavily indebted poor countries (HIPC)'],
  ['IBRD only'],
  ['IDA & IBRD total'],
  ['IDA total'],
  ['IDA blend'],
  ['IDA only'],
  ['Not classified'],
  ['Kyrgyz Republic'],
  ['St. Kitts and Nevis'],
  ['Latin America & Caribbean (excluding high income)'],
  ['Lao PDR'],
  ['Latin America & Caribbean'],
  ['Least developed countries: UN classification'],
  ['Low income'],
  ['Lower middle income'],
  ['Low & middle income'],
  ['Late-demographic dividend'],
  ['St. Martin (French part)'],
  ['Mid

In [14]:
with open(dir+'Output_Updated/'+'not_found.csv', 'w', encoding='UTF-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['name'])
    writer.writerows(not_found)
    # print(f"result: {result}")
    # print(isinstance(result, dict))

In [15]:
len(all_predictions),all_predictions

(289,
 {'Panama': 'panama',
  'Colombia': 'colombia',
  'South Korea': 'south-korea',
  'Peru': 'peru',
  'Costa Rica': 'costa-rica',
  'Oman': 'oman',
  'Dominican Republic': 'dominican-republic',
  'Bahrain': 'bahrain',
  'Guatemala': 'guatemala',
  'Nicaragua': 'nicaragua',
  'Honduras': 'honduras',
  'El Salvador': 'el-salvador',
  'Morocco': 'morocco',
  'Australia': 'australia',
  'Chile': 'chile',
  'Singapore': 'singapore',
  'Jordan': 'jordan',
  'Mexico': 'mexico',
  'Canada': 'canada',
  'Israel': 'israel',
  'United States': 'united-states',
  'China': 'china',
  'Japan': 'japan',
  'Germany': 'germany',
  'India': 'india',
  'United Kingdom': 'united-kingdom',
  'France': 'france',
  'Brazil': 'brazil',
  'Italy': 'italy',
  'Russia': 'russian-federation',
  'Spain': 'spain',
  'Indonesia': 'indonesia',
  'Turkey': 'turkey',
  'Netherlands': 'netherlands',
  'Saudi Arabia': 'saudi-arabia',
  'Switzerland': 'switzerland',
  'Argentina': 'argentina',
  'Sweden': 'sweden',
  

In [16]:
len(all_predictions_key),all_predictions_key

(289,
 {'Panama': 'PAN',
  'Colombia': 'COL',
  'South Korea': 'KOR',
  'Peru': 'PER',
  'Costa Rica': 'CRI',
  'Oman': 'OMN',
  'Dominican Republic': 'DOM',
  'Bahrain': 'BHR',
  'Guatemala': 'GTM',
  'Nicaragua': 'NIC',
  'Honduras': 'HND',
  'El Salvador': 'SLV',
  'Morocco': 'MAR',
  'Australia': 'AUS',
  'Chile': 'CHL',
  'Singapore': 'SGP',
  'Jordan': 'JOR',
  'Mexico': 'MEX',
  'Canada': 'CAN',
  'Israel': 'ISR',
  'United States': 'USA',
  'China': 'CHN',
  'Japan': 'JPN',
  'Germany': 'DEU',
  'India': 'IND',
  'United Kingdom': 'GBR',
  'France': 'FRA',
  'Brazil': 'BRA',
  'Italy': 'ITA',
  'Russia': 'RUS',
  'Spain': 'ESP',
  'Indonesia': 'IDN',
  'Turkey': 'TUR',
  'Netherlands': 'NLD',
  'Saudi Arabia': 'SAU',
  'Switzerland': 'CHE',
  'Argentina': 'ARG',
  'Sweden': 'SWE',
  'Poland': 'POL',
  'Belgium': 'BEL',
  'Thailand': 'THA',
  'Iran': 'IRL',
  'Austria': 'AUT',
  'Norway': 'NOR',
  'United Arab Emirates': 'ARE',
  'Nigeria': 'NGA',
  'South Africa': 'ZAF',
  'Hon

In [17]:

file_names=['fta_inforce','gdp_population','yearly_gdp_world']
for file in file_names:
    data=[]
    with open(dir+file+'.csv', mode="r",encoding='UTF-8', newline='') as csv_file: #"r" represents the read mode
        reader = csv.reader(csv_file) #this is the reader object
        count=0
        for item in reader:
            if count>0:
        # you have to loop through the document to get each data
                item.extend([all_predictions[item[0]],all_predictions_key[item[0]]])
                data.append(item)    
                # print(item)
            else:
                item.extend(['country_name_matched','country_key_matched'])
                header=item
                count+=1

    with open(dir+'Output_Updated/'+file+'_updated.csv', 'w', encoding='UTF-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)
        # print(f"result: {result}")
        # print(isinstance(result, dict))